In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import os
from config import NDVI_DATA_URL
from netCDF4 import Dataset
import io
import xarray as xr

In [ ]:
for year in range(1985, datetime.today().year + 1):
    print(year)
    days_count = 0
    if not str(year) in os.listdir(NDVI_DATA_URL):
        os.mkdir(f"{NDVI_DATA_URL}/{year}")
    r = requests.get(f"https://www.ncei.noaa.gov/data/land-normalized-difference-vegetation-index/access/{year}/")
    soup = BeautifulSoup(r.content, 'html.parser')
    tds = soup.find_all("td")
    files = []
    for td in tds:
        if td.text[-3:] == ".nc":
            files.append(td.text)
    for file in files:
        date = file.split("_")[4]
        file_req = requests.get(f"https://www.ncei.noaa.gov/data/land-normalized-difference-vegetation-index/access/{year}/{file}")
        file_req.raise_for_status()
        #select part of file to go from a total of 8 To to 6 Go :)
        buffer = io.BytesIO(file_req.content) #creates buffer from binary file
        dataset = xr.open_dataset(buffer, engine="h5netcdf")
        subset = dataset.sel( #select subset for france only to save a lot a space 
            longitude=slice(-5, 10),
            latitude=slice(51, 42))
        subset.to_netcdf(f"{NDVI_DATA_URL}/{year}/{date}.nc") #save new nc file
        
        days_count += 1
    print(days_count) #check numbers of days in month to be sure we have good data
    

1985


In [10]:
Dataset(f"{NDVI_DATA_URL}/1981/19810624.nc").variables["NDVI"].shape

(1, 180, 300)

In [19]:
Dataset(f"{NDVI_DATA_URL}/1981/out.nc").variables["NDVI"][:].shape

(1, 190, 295)